# load lib

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import time
import os
from tempfile import TemporaryDirectory
from PIL import Image
import logging
import csv
from tqdm import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [4]:
# def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
#     since = time.time()

#     # Create a temporary directory to save training checkpoints
#     with TemporaryDirectory() as tempdir:
#         best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')
    
#         torch.save(model.state_dict(), best_model_params_path)
#         best_acc = 0.0

#         for epoch in range(num_epochs):
#             print(f'Epoch {epoch}/{num_epochs - 1}')
#             print('-' * 10)

#             # Each epoch has a training and validation phase
#             for phase in ['train', 'val']:
#                 if phase == 'train':
#                     model.train()  # Set model to training mode
#                 else:
#                     model.eval()   # Set model to evaluate mode

#                 running_loss = 0.0
#                 running_corrects = 0

#                 # Iterate over data.
#                 for inputs, labels in dataloaders[phase]:
#                     inputs = inputs.to(device)
#                     labels = labels.to(device)

#                     # zero the parameter gradients
#                     optimizer.zero_grad()

#                     # forward
#                     # track history if only in train
#                     with torch.set_grad_enabled(phase == 'train'):
#                         outputs = model(inputs)
#                         _, preds = torch.max(outputs, 1)
#                         loss = criterion(outputs, labels)

#                         # backward + optimize only if in training phase
#                         if phase == 'train':
#                             loss.backward()
#                             optimizer.step()

#                     # statistics
#                     running_loss += loss.item() * inputs.size(0)
#                     running_corrects += torch.sum(preds == labels.data)
#                 if phase == 'train':
#                     scheduler.step()

#                 epoch_loss = running_loss / dataset_sizes[phase]
#                 epoch_acc = running_corrects.double() / dataset_sizes[phase]

#                 print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

#                 # deep copy the model
#                 if phase == 'val' and epoch_acc > best_acc:
#                     best_acc = epoch_acc
#                     torch.save(model.state_dict(), best_model_params_path)

#             print()

#         time_elapsed = time.time() - since
#         print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
#         print(f'Best val Acc: {best_acc:4f}')

#         # load best model weights
#         model.load_state_dict(torch.load(best_model_params_path, weights_only=True))
#     return model

In [28]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            val_loss = None  # Placeholder for validation loss

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                if phase == 'val':
                    val_loss = epoch_loss  # Store validation loss for scheduler

                    # deep copy the model
                    if epoch_acc > best_acc:
                        best_acc = epoch_acc
                        torch.save(model.state_dict(), best_model_params_path)

            # Update the learning rate for ReduceLROnPlateau
            if isinstance(scheduler, lr_scheduler.ReduceLROnPlateau):
                scheduler.step(val_loss)
            elif phase == 'train':
                scheduler.step()  # Standard step for other schedulers

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:.4f}')

        # Load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model


# RESNET50

## load base model

In [29]:
# Load the ResNet50 model
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)  # Replace the last layer with the number of classes
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## training model

In [30]:

learning_rate = 1e-4
weight_decay = 1e-2
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay) 
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5)

# Load data
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
}

data_dir = '/home/natthakit/304proj/pre_dataset'
image_datasets = {x: ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

# Train the model
trained_model = train_model(model, criterion, optimizer, scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.2613 Acc: 0.9083
val Loss: 0.1235 Acc: 0.9475

Epoch 1/24
----------
train Loss: 0.1120 Acc: 0.9606
val Loss: 0.0925 Acc: 0.9618

Epoch 2/24
----------
train Loss: 0.0722 Acc: 0.9767
val Loss: 0.0479 Acc: 0.9828

Epoch 3/24
----------
train Loss: 0.0474 Acc: 0.9858
val Loss: 0.0318 Acc: 0.9914

Epoch 4/24
----------
train Loss: 0.0486 Acc: 0.9860
val Loss: 0.0305 Acc: 0.9895

Epoch 5/24
----------
train Loss: 0.0387 Acc: 0.9883
val Loss: 0.0294 Acc: 0.9895

Epoch 6/24
----------
train Loss: 0.0319 Acc: 0.9893
val Loss: 0.0274 Acc: 0.9924

Epoch 7/24
----------
train Loss: 0.0340 Acc: 0.9891
val Loss: 0.0503 Acc: 0.9847

Epoch 8/24
----------
train Loss: 0.0223 Acc: 0.9930
val Loss: 0.0238 Acc: 0.9933

Epoch 9/24
----------
train Loss: 0.0165 Acc: 0.9954
val Loss: 0.0188 Acc: 0.9933

Epoch 10/24
----------
train Loss: 0.0302 Acc: 0.9909
val Loss: 0.0285 Acc: 0.9924

Epoch 11/24
----------
train Loss: 0.0285 Acc: 0.9905
val Loss: 0.0552 Acc: 0.9828

Ep

## save entire model and model's state_dict
> #### ***[!TIP ]*** 
> * Saving entire model: Keeps the architecture, making loading easier but might break across different PyTorch versions.
> * Saving state dict: Preferred for flexibility and compatibility.

In [25]:
!rm -rf 'saved/crop_resnet50_state_dict.pth'
!rm -rf 'saved/crop_resnet50_full_model.pth'


In [31]:
# Save the entire model
torch.save(trained_model, 'saved/pre_resnet50_full_model.pth')

# Save only the model's state_dict (recommended)
torch.save(trained_model.state_dict(), 'saved/pre_resnet50_state_dict.pth')

# SWINV2_s

## load base model

In [9]:
model = models.swin_v2_s(weights="IMAGENET1K_V1")
num_ftrs = model.head.in_features
model.head = nn.Linear(num_ftrs, 4)  # Replace the last layer with the number of classes
model = model.to(device)

Downloading: "https://download.pytorch.org/models/swin_v2_s-637d8ceb.pth" to /home/natthakit/.cache/torch/hub/checkpoints/swin_v2_s-637d8ceb.pth
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 191M/191M [00:12<00:00, 16.1MB/s]


## training model

In [12]:
# Define criterion, optimizer, and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Load data
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
data_dir = '/home/natthakit/304proj/dataset'
image_datasets = {x: ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

# Train the model
trained_model = train_model(model, criterion, optimizer, scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.5863 Acc: 0.7730
val Loss: 0.3265 Acc: 0.8827

Epoch 1/24
----------
train Loss: 0.3330 Acc: 0.8743
val Loss: 0.1675 Acc: 0.9307

Epoch 2/24
----------
train Loss: 0.2551 Acc: 0.9037
val Loss: 0.1992 Acc: 0.9322

Epoch 3/24
----------
train Loss: 0.2068 Acc: 0.9230
val Loss: 0.1482 Acc: 0.9566

Epoch 4/24
----------
train Loss: 0.1888 Acc: 0.9291
val Loss: 0.0961 Acc: 0.9596

Epoch 5/24
----------
train Loss: 0.1659 Acc: 0.9380
val Loss: 0.1060 Acc: 0.9566

Epoch 6/24
----------
train Loss: 0.1554 Acc: 0.9424
val Loss: 0.0815 Acc: 0.9711

Epoch 7/24
----------
train Loss: 0.1225 Acc: 0.9561
val Loss: 0.0659 Acc: 0.9772

Epoch 8/24
----------
train Loss: 0.1180 Acc: 0.9589
val Loss: 0.0623 Acc: 0.9802

Epoch 9/24
----------
train Loss: 0.1151 Acc: 0.9561
val Loss: 0.0611 Acc: 0.9772

Epoch 10/24
----------
train Loss: 0.1132 Acc: 0.9576
val Loss: 0.0614 Acc: 0.9794

Epoch 11/24
----------
train Loss: 0.1071 Acc: 0.9601
val Loss: 0.0564 Acc: 0.9810

Ep

## Save model

In [17]:
# Save the entire model
torch.save(trained_model, 'saved/swinv2_s_full_model.pth')

# Save only the model's state_dict (recommended)
torch.save(trained_model.state_dict(), 'saved/swinv2_s_state_dict.pth')

# MaxVit_T

## load model

In [49]:
# Import the MaxVit model and weights from torchvision
from torchvision.models import maxvit_t, MaxVit_T_Weights

# Load the MaxVit-T model with pre-trained weights
weights = MaxVit_T_Weights.IMAGENET1K_V1
model = maxvit_t(weights=weights)

# Check the last layer in the classifier to get the number of input features
# Typically, this will be the last layer within the classifier's Sequential block
num_ftrs = model.classifier[-1].in_features  # Access the final layer in the classifier
model.classifier[-1] = nn.Linear(num_ftrs, 4)  # Replace with a new linear layer for 4 classes
model = model.to(device)

# Define the inference transforms
inference_transforms = weights.transforms()

In [63]:
inference_transforms

ImageClassification(
    crop_size=[224]
    resize_size=[224]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

## training model

In [59]:
# Define criterion, optimizer, and scheduler

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)  # Use Adam optimizer with lr=1e-4
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': inference_transforms,
}

# Load data as before
image_datasets = {x: ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

trained_model = train_model(model, criterion, optimizer, scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.0996 Acc: 0.9652
val Loss: 0.0564 Acc: 0.9825

Epoch 1/24
----------
train Loss: 0.0903 Acc: 0.9659
val Loss: 0.0278 Acc: 0.9939

Epoch 2/24
----------
train Loss: 0.0956 Acc: 0.9660
val Loss: 0.0420 Acc: 0.9863

Epoch 3/24
----------
train Loss: 0.0738 Acc: 0.9723
val Loss: 0.0308 Acc: 0.9901

Epoch 4/24
----------
train Loss: 0.0691 Acc: 0.9743
val Loss: 0.0277 Acc: 0.9924

Epoch 5/24
----------
train Loss: 0.0806 Acc: 0.9718
val Loss: 0.0234 Acc: 0.9939

Epoch 6/24
----------
train Loss: 0.0649 Acc: 0.9779
val Loss: 0.0336 Acc: 0.9909

Epoch 7/24
----------
train Loss: 0.0602 Acc: 0.9783
val Loss: 0.0212 Acc: 0.9947

Epoch 8/24
----------
train Loss: 0.0428 Acc: 0.9853
val Loss: 0.0158 Acc: 0.9970

Epoch 9/24
----------
train Loss: 0.0509 Acc: 0.9816
val Loss: 0.0163 Acc: 0.9970

Epoch 10/24
----------
train Loss: 0.0485 Acc: 0.9825
val Loss: 0.0169 Acc: 0.9970

Epoch 11/24
----------
train Loss: 0.0437 Acc: 0.9835
val Loss: 0.0168 Acc: 0.9962

Ep

In [10]:
# Load the entire model
model = torch.load('saved/maxvit_T_full_model.pth')
model = model.to(device)  # Move to the correct device

In [12]:
# Define criterion, optimizer, and scheduler
data_dir = '/home/natthakit/304proj/Coursera-Content/Brain-MRI/'
image_datasets = {x: ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)  # Use Adam optimizer with lr=1e-4
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(224, interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

# Load data as before
image_datasets = {x: ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

trained_model = train_model(model, criterion, optimizer, scheduler, num_epochs=10)

Epoch 0/9
----------
train Loss: 0.1497 Acc: 0.9455
val Loss: 0.2393 Acc: 0.9500

Epoch 1/9
----------
train Loss: 0.1180 Acc: 0.9587
val Loss: 0.0712 Acc: 0.9500

Epoch 2/9
----------
train Loss: 0.1276 Acc: 0.9550
val Loss: 0.0315 Acc: 1.0000

Epoch 3/9
----------
train Loss: 0.1032 Acc: 0.9636
val Loss: 0.0510 Acc: 1.0000

Epoch 4/9
----------
train Loss: 0.0918 Acc: 0.9709
val Loss: 0.0523 Acc: 0.9500

Epoch 5/9
----------
train Loss: 0.1081 Acc: 0.9626
val Loss: 0.0258 Acc: 1.0000

Epoch 6/9
----------
train Loss: 0.1002 Acc: 0.9651
val Loss: 0.0233 Acc: 1.0000

Epoch 7/9
----------
train Loss: 0.0845 Acc: 0.9703
val Loss: 0.0262 Acc: 1.0000

Epoch 8/9
----------
train Loss: 0.0728 Acc: 0.9740
val Loss: 0.0234 Acc: 1.0000

Epoch 9/9
----------
train Loss: 0.0652 Acc: 0.9767
val Loss: 0.0272 Acc: 1.0000

Training complete in 4m 35s
Best val Acc: 1.000000


In [13]:
torch.save(trained_model, 'saved/new_maxvit_T_full_model.pth')

# Save only the model's state_dict (recommended)
torch.save(trained_model.state_dict(), 'saved/new_maxvit_T_state_dict.pth')

## Save model

In [60]:
torch.save(trained_model, 'saved/maxvit_T_full_model.pth')

# Save only the model's state_dict (recommended)
torch.save(trained_model.state_dict(), 'saved/maxvit_T_state_dict.pth')

# EfficientNet_B0

## load base model

In [6]:
# Load EfficientNet_B0 model with pretrained weights
model = models.efficientnet_v2_s(weights='IMAGENET1K_V1')
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 4)  # Replace the last layer with the number of classes
model = model.to(device)


## training model

In [7]:
# Define criterion, optimizer, and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [8]:
# Load data with EfficientNet preprocessing transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256, interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

# Set up datasets and dataloaders as before
data_dir = '/home/natthakit/304proj/dataset'
image_datasets = {x: ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

# Train the model
trained_model = train_model(model, criterion, optimizer, scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.7969 Acc: 0.7105
val Loss: 0.4372 Acc: 0.8417

Epoch 1/24
----------
train Loss: 0.3514 Acc: 0.8734
val Loss: 0.2527 Acc: 0.9038

Epoch 2/24
----------
train Loss: 0.2577 Acc: 0.9020
val Loss: 0.1470 Acc: 0.9508

Epoch 3/24
----------
train Loss: 0.2093 Acc: 0.9219
val Loss: 0.1064 Acc: 0.9621

Epoch 4/24
----------
train Loss: 0.1735 Acc: 0.9377
val Loss: 0.0904 Acc: 0.9682

Epoch 5/24
----------
train Loss: 0.1591 Acc: 0.9436
val Loss: 0.0614 Acc: 0.9811

Epoch 6/24
----------
train Loss: 0.1314 Acc: 0.9517
val Loss: 0.0513 Acc: 0.9864

Epoch 7/24
----------
train Loss: 0.1233 Acc: 0.9552
val Loss: 0.0508 Acc: 0.9864

Epoch 8/24
----------
train Loss: 0.1214 Acc: 0.9550
val Loss: 0.0429 Acc: 0.9917

Epoch 9/24
----------
train Loss: 0.1292 Acc: 0.9534
val Loss: 0.0423 Acc: 0.9902

Epoch 10/24
----------
train Loss: 0.1330 Acc: 0.9524
val Loss: 0.0445 Acc: 0.9894

Epoch 11/24
----------
train Loss: 0.1299 Acc: 0.9554
val Loss: 0.0395 Acc: 0.9894

Ep

## Save model

In [9]:
torch.save(trained_model, 'saved/eff_s_full_model.pth')

# Save only the model's state_dict (recommended)
torch.save(trained_model.state_dict(), 'saved/eff_s_state_dict.pth')